# Sampling for Bayesian models

## Takeaways and objectives from notebooks 3a-3e

Sampling is a non-trivial approach to model inference and parameter estimation.  We will build our understanding of sampling algorithms from scratch starting with how to compute expectations of functions from distributions from which we don't know how to draw samples (using **importance sampling**) and then building on these ideas with **rejection sampling** which introduces the key idea of not accepting all samples that are proposed.

Next, we intuitively motivate the need for Markov Chain Monte Carlo by observing that even in a two-dimensional model, it is difficult to hit the high-probability regions of the parameters.  We will finish with sampling from multi-dimensional models using the **Metropolis algorithm**.  Each step will be supported with exercises and code. We will examine the advantages and drawbacks of each method.

Finally, we will briefly touch on modern MCMC methods which resolve some of the drawbacks including HMC, ADVI and NUTS.

## Bayesian analysis requires solution of integrals

When we observe data $y$ that we hypothesize as being obtained from a sampling model $f(y|\theta)$, where $\theta$ is a vector of (unknown) model parameters, a Bayesian places a *prior* distribution $p(\theta)$ on the parameters to describe the uncertainty in the true values of the parameters. Bayesian inference, then, is obtained by calculating the *posterior* distribution, which is proportional to the product of these quantities:

$$p(\theta | y) \propto f(y|\theta) p(\theta),$$

since evaluating the complete equation:

$$p(\theta | y) = \frac{f(y|\theta) p(\theta)}{f(y)}$$

is typically intractable.  In fact for most problems of interest, the normalizing constant $f(y)$ cannot be calculated because it involves mutli-dimensional integration over $\theta$.

Let us assume that $h(\theta)$ is an interesting function of the unknown parameter (variance, expected value, ...), then the resulting expectation is:

$$E[h(\theta)|y] = \int h(\theta) p(\theta|y) d\theta \approx \frac{1}{n}\sum_{i=1}^n h(\theta_i)$$

We also need integration to obtain marginal estimates from a joint probability distribution. If $\theta$ is of length $K$, then inference about any particular parameter is obtained by:

$$p(\theta_i|y) \propto \int p(\theta|y) d\theta_{-i}$$

where the `-i` subscript indicates all elements except the $i^{th}$.

The problem of inference in Bayesian analysis is largely reduced to that of computing integrals such as those above.

### Simulation Approaches for Bayesian Computation (and solving integrals)

Since analysis is off the table, a reasonable alternative is to attempt to estimate the integral using numerical methods. For example, consider the expected value of a random variable $\mathbf{x}$:

$$\begin{gathered}
\begin{split}E[{\bf x}] = \int {\bf x} f({\bf x}) d{\bf x}, \qquad
{\bf x} = \{x_1,...,x_k\}\end{split}\notag\\\begin{split}\end{split}\notag\end{gathered}$$

where $f({\bf x})$ is the probability density of $\bf x$ and $k$ (the dimension of vector $x$) is perhaps very large. If we can produce a reasonable number of random vectors $\{{\bf x_i}\}$, we can use these values to approximate the unknown integral. This process is known as *Monte Carlo integration*. In general, MC integration allows integrals against probability density functions:

$$\begin{gathered}
\begin{split}I = \int h(\mathbf{x}) f(\mathbf{x}) \mathbf{dx}\end{split}\notag\\\begin{split}\end{split}\notag\end{gathered}$$

to be estimated by finite sums:

$$\begin{gathered}
\begin{split}\hat{I} = \frac{1}{n}\sum_{i=1}^n h(\mathbf{x}_i),\end{split}\notag\\\begin{split}\end{split}\notag\end{gathered}$$

where $\mathbf{x}_i$ is a sample from $f$. This estimate is valid and useful because:

-   By the strong law of large numbers:

$$\begin{gathered}
\begin{split}\hat{I} \rightarrow I   \mbox{   with probability 1}\end{split}\notag\\\begin{split}\end{split}\notag\end{gathered}$$

-   Simulation error can be measured and controlled:

$$Var(\hat{I}) = \frac{1}{n(n-1)}\sum_{i=1}^n
   (h(\mathbf{x}_i)-\hat{I})^2$$

Note that to perform the above computations, we need to be able to draw samples from the density $f({\bf x})$, which is typically not the case, so the analysis is theoretical but cannot be applied as-is in practice.

#### Notebook 3b
In this notebook, we shall see how to estimate the above when we cannot sample from $f(x)$.  To this end we will design a distribution $q(x)$ from which we know how to sample.

#### Notebook 3c
Further, we will see how to build a sampler from such a distribution $f({\bf x})$ so that we can draw random samples from the distribution $f({\bf x})$.  Note that this is in some sense more powerful that being able to only compute expectations.

The above shows that we can apply the techniques we discussed to Bayesian models by simply considering the likelihood and prior product $f(y|\theta) p(\theta)$ to be the density of interest. 

## Motivation for Markov Chain Monte Carlo
However, even with the above techniques, sampling from multidimensional distributions becomes hard pretty quickly in the sense that many samples have very low probability density.  Thus it will take a long time for our sampler to converge.  This is discussed for example in [Ian Murray's talk](http://videolectures.net/mlss09uk_murray_mcmc/)[1].

Markov Chain Monte Carlo tries to attack the problem by constructing a biased random walk that preferentially stays in regions of high density so that once we get a good sample, we tend to explore the region around it. This helps us sample the distribution in a more effective way.

#### Notebook 3d
We will explore this motivation for Markov Chain constructions instead of pure Monte Carlo approaches. We will also examine the Metropolis method, code it and understand its failure modes.

#### Notebook 3e
We will learn about advanced algorithms for MCMC and study their performance.

## A historical note
The history of Monte Carlo is actually tied to Los Alamos in the time of the second world war and to the development of nuclear weapons themselves.  The first paper on the idea is due to Metropolis and Ulam [2], although Metropolis himself states that Enrico Fermi was using the Monte Carlo technique in the 30s in Italy to predict the results of experiments when he couldn't sleep at night [3].

The historical note [3] is a great and enjoyable read, highly recommended.

## References
1. Ian Murray, Markov Chain Monte Carlo, Videolecture.net [talk](http://videolectures.net/mlss09uk_murray_mcmc/) for slides and [youtube](https://www.youtube.com/watch?v=hn0Vfzcnmig) for the talk itself.
2. Nicholas Metropolis and S. Ulam. [The Monte Carlo Method](https://www.jstor.org/stable/2280232). Journal of the American Statistical Association. Vol. 44, No. 247 (Sep., 1949), pp. 335-341
3. Nicholas Metropolis. [The Beginning of the Monte Carlo method](http://library.lanl.gov/cgi-bin/getfile?00326866.pdf). Los Alamos Science. 1987, pp. 125-130.
4. Cam Davidson-Pilon. [The greatest theorem never told](http://nbviewer.jupyter.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter4_TheGreatestTheoremNeverTold/Ch4_LawOfLargeNumbers_PyMC3.ipynb). In [Probabilistic Programming & Bayesian Methods for Hackers](http://camdavidsonpilon.github.io/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/).